In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import datetime as dt
import pickle
from utils import fractional_years
from predcit import Predictor
pd.set_option("display.max_columns", None)

## Load Data

In [ ]:
data_path = '../data'
df_raw = pd.read_excel(f'{data_path}/inscriptos_2018_4_6.xlsx')

### Enrich with estimated bday from DNI

In [ ]:
model_info = {"location": "filesystem", "path": "./", "filename":"mega_spline_model.pickle"}
bday_predictor = Predictor(model_info)
df_raw['estimated_bday'] = np.array(bday_predictor.predict(df_raw.nrodocumento.values.reshape(-1, 1)))

In [ ]:
df_raw['estimated_age'] = (df_raw.FechaCpte - df_raw.estimated_bday).apply(lambda td: td.days/365.25)

### Enrich with foreigner from DNI

In [ ]:
df_raw['foreigner'] = df_raw.nrodocumento>90e6

### Enrich with course start date

In [ ]:
df_start_date = pd.read_csv(f'{data_path}/fecha_inicio_clases.csv')
df_start_date['end_inscr_date'] = df_start_date.fecha_fin_insripcion_estimada.apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))
df_start_date['start_course_date'] = df_start_date.fecha_inicio_cursada.apply(lambda x: dt.datetime.strptime(x,'%d/%m/%Y'))
df_start_date.drop(columns=['fecha_fin_insripcion_estimada','fecha_inicio_cursada','Unnamed: 3'],inplace=True)

In [ ]:
df_raw = df_raw.merge(df_start_date, on='anioperi')
df_raw['start_course_to_inscr_days'] = ((df_raw['start_course_date']-df_raw['FechaCpte'])/(24*60*60*1e9)).astype(int)
df_raw['end_inscr_to_inscr_days'] = ((df_raw['end_inscr_date']-df_raw['FechaCpte'])/(24*60*60*1e9)).astype(int)
df_raw.drop(columns=['end_inscr_date','start_course_date'],inplace=True)

In [ ]:
df_raw

### Enrich with history
Past churns, past completed courses, proff historic churn


In [ ]:
df_raw[df_raw.Anio_id==2022].bajadeasistencia.value_counts()

In [ ]:
df_raw['year'] = df_raw['anioperi'].apply(lambda x: int(x.split('-')[0]))
df_raw['period'] = df_raw['anioperi'].apply(lambda x: int(x.split('-')[1]))
df_enr = df_raw.merge(df_raw,on='nrodocumento')

In [ ]:
c1 = (df_enr['year_x']==df_enr['year_y'])&(df_enr['period_x']>df_enr['period_y'])
c2 = df_enr['year_x']>df_enr['year_y']
df_enr = df_enr[c1|c2]

In [ ]:
df_enr.tail()#[df_enr.nrodocumento==36406220.0]

In [ ]:
df_enr[df_enr.nrodocumento==42496842.0]

### Enrich with convenio

In [ ]:
df_raw['con_convenio'] = df_raw.convenio!=df_raw.convenio.value_counts().index[0]

### Drop Incomplete semester

In [ ]:
df_raw = df_raw[df_raw['Anio_id']<2022]

### Drop unused columns

In [ ]:
drop_cols_1 = ["anioperi","cursointerno_id","FechaCpte","nrodocumento","Alumno_id","Sede_id","nombre","Anio_id",
"descnivel","cursodenivel","esidiomaonline","descsedefis","pais_id","nombpais","provincia_id","nombpcia","localidad_ubi","FechaNacimiento"]

In [ ]:
drop_cols_2 = ["edad",  # dirty column replaced by estimated_age  
               "estimated_bday", # estimated_age has the relevant info
               "nivel_id",  # Combination of "idioma_id","grado","TipoDeNivel"
              ]

In [ ]:
drop_cols_1.extend(drop_cols_2)

### Columns with high cardinality

In [ ]:
high_cardinality_cols = ["convenio","esidiomaic","nombprof"]

In [ ]:
df = df_raw.drop(columns=drop_cols_1)

### Try with LightGBM - NO One Hot Encoding

In [ ]:
# Set type of categorical variables so LGBM can predict
is_object = df.dtypes=='object'
for feat in is_object[is_object==True].index.values:
    df[feat] = df[feat].astype('category')
    
int_attrs = ['Periodo_id','grado','bajadeasistencia','esidiomaic','EsUba','Nuevo']
for attr in int_attrs:
    df[attr] = df[attr].astype(int)
    
scale_cols = ["start_course_to_inscr_days","estimated_age","end_inscr_to_inscr_days"]
label_encoded_cols = ["Descripcion","idioma_id","deschora","convenio","TipoDeNivel","Sexo","Cursada_id","nombprof"]

for attr in label_encoded_cols:
    df[attr] = df[attr].astype(str)
    df[attr] = df[attr].astype(str)
    
all_columns = list(df.columns.values)
all_columns.remove('bajadeasistencia')

In [ ]:
class columnDropperTransformer():
    def __init__(self,columns):
        self.columns=columns

    def transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

    def fit(self, X, y=None):
        return self 

In [ ]:
# Split Train and Test
label = 'bajadeasistencia'
y = df[label].astype('int')
X = df.drop(columns=[label])
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder, OneHotEncoder
from lightgbm import LGBMClassifier
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier

def full_model(remaining_cols, all_cols):
    xgmodel = XGBClassifier(eta=0.3, gamma=0.2, max_depth=5, subsample=0.5,reg_alpha=10,reg_lambda=0)
    lgbm = LGBMClassifier(min_data_in_leaf=10,max_depth=-2,num_leaves=10,)

    drop_cols = [c for c in all_cols if c not in remaining_cols]

    present_label_encoded_cols = [c  for c in label_encoded_cols if c not in drop_cols]
    present_scale_cols = [c  for c in scale_cols if c not in drop_cols]

    ct = ColumnTransformer(
    transformers=[
        ("scaler", StandardScaler(), present_scale_cols),
        ("label_encoder", OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1), present_label_encoded_cols)
        #("1HotEncoder", OneHotEncoder(drop='if_binary',handle_unknown='infrequent_if_exist',min_frequency=0.05), present_label_encoded_cols)
    ],
    remainder='passthrough'
    )
    pipe = Pipeline([
                ('columnDropper', columnDropperTransformer(drop_cols)),
                ('column_scaler', ct),
                ('lgbm', lgbm)
                #('xgb', xgmodel)
                ])
    return pipe


In [ ]:
# Add benchmark
pipe = full_model(remaining_cols=all_columns, all_cols=all_columns)
pipe.fit(X_train,y_train)
max_auc = roc_auc_score(y_test, pipe.predict_proba(X_test)[:,1])

auc = 0.0
fixed_cols = []

# Add best feature one at a time
while auc<.99:
    attr_score = {}
    for attr in [c for c in all_columns if c not in fixed_cols]:
        remaining_cols = fixed_cols+[attr]
        pipe = full_model(remaining_cols=remaining_cols, all_cols=all_columns)
        pipe.fit(X_train,y_train)
        attr_score[tuple(remaining_cols)] = roc_auc_score(y_test, pipe.predict_proba(X_test)[:,1]) / max_auc

    #Sort by score
    attrs, auc = sorted(attr_score.items(), key=lambda item: item[1], reverse=True)[0]
    fixed_cols = list(attrs) # Add best feature
    print(fixed_cols, auc)